In [1]:
import pickle
from collections import Counter
from datetime import time, datetime
import re

import attr
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from pprint import pprint

import wrestling

### This noteboook is for vizzes, BUT it has some great logical flow from top to bottom for how to generate/create the match objects!  Take these functions (proprietary) and use them as parsers with more!

In [2]:
wrestling

<module 'wrestling' from '/Users/nick/PycharmProjects/wrestling/wrestling/__init__.py'>

In [3]:
with open('matches.p', 'rb') as f:
    matches = pickle.load(f)
print(len(matches))

794952


In [4]:
m1 = matches[0]
m2 = matches[100]
del matches

In [5]:
wrestling.Event(id='fafa', name='ankfjsnl', kind=wrestling.Mark('Tournament'))

Event(name='Ankfjsnl', _kind=Tournament)

In [6]:
# exploring events
with open('events.p', 'rb') as f:
    events = pickle.load(f) 

print(len(events))

events[0]['id'] = events[0].pop('id')
events[0]['name'] = events[0].pop('name')
events[0]['kind'] = wrestling.Mark(events[0].pop('type'))

EV = wrestling.Event(**events[0])
del events
EV

53791


Event(name='Cowboy Open', _kind=Tournament)

In [7]:
# proprietary function
def parse_api_scoring_events(event_string):
    #! this function will be large and more incremental and will call other validation functions since
    # the TS is primarily where we get our data from it needs to be accurate but its also where most of the errors are
    
    # split into list of individual events then type convert
    # use nested to avoid storing temp lists in memory
    return ((int(x[0]), int(x[1]), int(x[2]) + 1, x[3], x[4].lower()) for x in [x.split('*') for x in event_string.split('#')])

In [8]:
m1['scoringEvents']

'3*3*1*DEFER*Red#3*3*1*BOT*Green#4*26*1*E1*Green#5*5*2*BOT*Red#5*14*2*E1*Red#7*0*2*WS*Red'

In [9]:
FOC = wrestling.Wrestler(name='nick, anthony', team='eagles')
OPP = wrestling.Wrestler(name='mich, duo', team='sea hawks')

nice_ts = parse_api_scoring_events(m1['scoringEvents'])
# add START event
ts1 = []
for sc in nice_ts:
    action = wrestling.CollegeScoring(
        focus_color='red',
        initiator=sc[-1],
        time_stamp=time(minute=sc[0], second=sc[1]),
        label=wrestling.base.CollegeLabel(sc[-2]),
        period=sc[2],
    )
    ts1.append(action)
pprint(ts1)

[CollegeScoring(time_stamp=datetime.time(0, 3, 3), initiator='red', label=DEFER),
 CollegeScoring(time_stamp=datetime.time(0, 3, 3), initiator='green', label=BOT),
 CollegeScoring(time_stamp=datetime.time(0, 4, 26), initiator='green', label=E1),
 CollegeScoring(time_stamp=datetime.time(0, 5, 5), initiator='red', label=BOT),
 CollegeScoring(time_stamp=datetime.time(0, 5, 14), initiator='red', label=E1),
 CollegeScoring(time_stamp=datetime.time(0, 7), initiator='red', label=WS)]


In [10]:
nice_ts = parse_api_scoring_events(m2['scoringEvents'])

# add START event
ts2 = []
for sc in nice_ts:
    action = wrestling.CollegeScoring(
        focus_color='red',
        initiator=sc[-1],
        time_stamp=time(minute=sc[0], second=sc[1]),
        label=wrestling.base.CollegeLabel(sc[-2]),
        period=sc[2]
    )
    ts2.append(action)
pprint(ts2)

[CollegeScoring(time_stamp=datetime.time(0, 1, 11), initiator='red', label=T2),
 CollegeScoring(time_stamp=datetime.time(0, 2, 54), initiator='red', label=BOT),
 CollegeScoring(time_stamp=datetime.time(0, 3, 11), initiator='red', label=E1),
 CollegeScoring(time_stamp=datetime.time(0, 4, 23), initiator='red', label=T2),
 CollegeScoring(time_stamp=datetime.time(0, 4, 55), initiator='green', label=BOT)]


In [11]:
regex = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"

match1 = wrestling.CollegeMatch(
    id=m1['matchID'],
    event=EV,
    date_=datetime.strptime(re.search(regex, m1['matchID']).group(1), "%Y-%m-%d %H:%M:%S"),  # make into func
    result=wrestling.base.Result.WD,  # need result parsing func
    overtime=False,  # can be set to optional and false by default
    focus=FOC,  # use parse name function here
    opponent=OPP,  # use parse name function here
    weight=wrestling.base.Mark(m1['theWeightClass']),  # could have converter, but idk why -- users should just typecast to int
    time_series=tuple(sorted(ts1))
)

match2 = wrestling.CollegeMatch(
    id=m2['matchID'],
    event=EV,
    date_=datetime.strptime(re.search(regex, m2['matchID']).group(1), "%Y-%m-%d %H:%M:%S"),  # make into func
    result=wrestling.base.Result.WT,  # need result parsing func
    overtime=False,  
    focus=FOC,  # use parse name function here
    opponent=OPP,  # use parse name function here
    weight=wrestling.base.Mark(m2['theWeightClass']),  
    time_series=tuple(sorted(ts2))
)

match3 = wrestling.CollegeMatch(
    id=m2['matchID'],
    event=EV,
    date_=datetime.strptime(re.search(regex, m2['matchID']).group(1), "%Y-%m-%d %H:%M:%S"),  # make into func
    result=wrestling.base.Result.LM,  # need result parsing func
    overtime=False,  
    focus=FOC,  # use parse name function here
    opponent=OPP,  # use parse name function here
    weight=wrestling.base.Mark(m2['theWeightClass']),  
    time_series=tuple(sorted(ts2))
)

pprint(match1)
pprint(match2)
match3.weight

CollegeMatch(event=Cowboy Open, result=Win Dec, focus=Nick, Anthony, opponent=Mich, Duo, _weight=125, duration=420, time_series=6 actions)
CollegeMatch(event=Cowboy Open, result=Win Tech, focus=Nick, Anthony, opponent=Mich, Duo, _weight=125, duration=420, time_series=5 actions)


'125'

In [12]:
# proprietary
# modifies ts_dict in-place
def add_ts_points(ts_dict):
    for i, d in enumerate(ts_dict):
        if i == 0:
            ts_dict[i]['focus_score'] = 0
            ts_dict[i]['opp_score'] = 0
            continue
        if d['str_label'].startswith('f'):    
            ts_dict[i]['focus_score'] = ts_dict[i]['label'].value + ts_dict[i - 1]['focus_score']
            ts_dict[i]['opp_score'] = ts_dict[i - 1]['opp_score']
        elif d['str_label'].startswith('o'):    
            ts_dict[i]['focus_score'] = ts_dict[i - 1]['focus_score']
            ts_dict[i]['opp_score'] = ts_dict[i]['label'].value + ts_dict[i - 1]['opp_score']
        else:
            raise ValueError(f"Invalid `str_label`, expected startswith = 'o' or 'f', got {d['str_label']}")
    return ts_dict

In [14]:
match_dict1 = match1.to_dict()
ts_dict1 = add_ts_points(match1.to_dict(ts_only=True)[1:])

match_dict2 = match2.to_dict()
ts_dict2 = add_ts_points(match2.to_dict(ts_only=True)[1:])
ts_dict2[0]

{'time': '02:54',
 'period': 2,
 'str_label': 'fBOT',
 'label': CollegeLabel(tag='BOT', isvalid=True),
 'focus_name': 'Nick, Anthony',
 'opp_name': 'Mich, Duo',
 'event_name': 'Cowboy Open',
 'focus_score': 0,
 'opp_score': 0}

In [17]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [26]:
def draw_results_hist(match_list, template):
    df = pd.DataFrame.from_records((m.to_dict(results_only=True) for m in match_list))
    return px.histogram(
        df,
        y="method",
        histfunc="count",
        color="binary",
        barmode="group",
        category_orders=dict(
            method=["Dec", "Major", "Tech", "Fall", "Contest"],
            result=["Win", "Loss", "No"],
        ),
        color_discrete_sequence=["gold", "gray"],
        labels=dict(method="Method", result="Result"),
        template=template,
        title=template,
    ).update_layout(
        xaxis_title="Count",
        yaxis_title="Result",
        legend=dict(xanchor="center", yanchor="top", x=0.5, y=1.0),
        legend_orientation="h",
        legend_title_text=None,
    )


In [27]:
for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
    fig = draw_results_hist([match1, match2, match3], template)
    fig.show()